In [1]:
import os
import sys
from pathlib import Path
module_path = Path('.').absolute().parents[4]
if str(module_path) not in sys.path:
    sys.path.append(str(module_path))
    
from datetime import datetime
import plotly.graph_objs as go
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from capstone.quant.infrastructure.df_curve_data_container import DFCurveRequest, JPDFCurveDataSource
from capstone.quant.infrastructure.forward_rate_data_container import ForwardRateRequest, JPForwardRateDataSource, ForwardRateFlatFileDataSource
from capstone.quant.infrastructure.rate_vol_data_container import RateVolRequest, JPRateVolDataSource, RateVolFlatFileDataSource
from capstone.quant.backtest.functions.rolling_swaptions import rolling_swaptions
from capstone.quant.backtest.strategies.rolling_swaptions import RollingSwaptions, RollingSwaptionsState
from capstone.quant.tradable.swaption import Swaption
from capstone.quant.tradable.forwardstartswap import ForwardStartSwap

***Rolling Swaption***

In [2]:
start_date = datetime(2007, 3, 30)
end_date = datetime(2024, 7, 31)

calendar = ['NYC', 'LON']
currency = 'USD'

legs = {
            'leg1': {
                'currency': currency,
                'style': 'Receiver',
                'expiry': '1y',
                'strike': -150,
                'strike_type': 'spot',
                'tenor': '10y',
                'sizing': 1 / 6.0,
                'delta_hedge': "Unhedged",
            },
        }

tranches = { 
            'm1': {
                'roll_months': [1,7],
                'roll_day': 15,
            },
            'm2': {
                'roll_months': [2,8],
                'roll_day': 15,
            },
            'm3': {
                'roll_months': [3,9],
                'roll_day': 15,
            },
            'm4': {
                'roll_months': [4,10],
                'roll_day': 15,
            },
            'm5': {
                'roll_months': [5,11],
                'roll_day': 15,
            },
            'm6': {
                'roll_months': [6,12],
                'roll_day': 15,
            },
        }

tc_rate = {Swaption: 0.1, ForwardStartSwap: 0.0002}

results, pnl_series, portfolio_expanded = rolling_swaptions(
        start_date, end_date, calendar, currency,
        legs, tranches,
        tc_rate,
    
        rate_vols_data_source=JPRateVolDataSource(), 
#        rate_vols_data_source=RateVolFlatFileDataSource('/misc/Traders/Solutions/backtests/data_cache/rates/gbp_vol_manual_expanded.csv'),
    
        fwd_rates_data_source=JPForwardRateDataSource(),
#        fwd_rates_data_source=ForwardRateFlatFileDataSource('/misc/Traders/Solutions/backtests/data_cache/rates/gbp_forward_rates_manual_expanded.csv'),   
    
        spot_rates_data_source=JPDFCurveDataSource(),
        df_rates_data_source=JPDFCurveDataSource(),
        df_rates_type='OIS',
)

/home/ychu/github/quant-research/capstone/quant/data/Datalake3.py:242: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return ret.append(self.getData(source, ticker, fields, mid_plus_1_sec, end))
/home/ychu/github/quant-research/capstone/quant/data/Datalake3.py:242: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return ret.append(self.getData(source, ticker, fields, mid_plus_1_sec, end))
/home/ychu/github/quant-research/capstone/quant/data/Datalake3.py:242: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return ret.append(self.getData(source, ticker, fields, mid_plus_1_sec, end))
/home/ychu/github/quant-research/capstone/quant/data/Datalake3.py:242: FutureWarning: The frame.append method is deprecated and will be removed from pa

UnboundLocalError: local variable 'data_df' referenced before assignment

In [ ]:
records = [{'date': x.time_stamp, 'forward_rate': x.legs_info['leg1'][list(tranches.keys())[0]]['forward_rate']} for x in results]
forward_rate_series = pd.DataFrame.from_dict(records)

fig, ax = plt.subplots(figsize=(12,4))
my_title = 'Strategy performance'
ax.plot(pnl_series['date'], pnl_series['pnl'])
plt.title(my_title)
ax.margins(x=0)
ax.legend(['Hedge (LHS)'], loc='upper center', frameon=False)
ax2 = ax.twinx()
ax2.plot(forward_rate_series['date'], forward_rate_series['forward_rate'], color='lightblue')
ax2.legend(['Fwd Rate (RHS)'], frameon=False)
plt.show()

***Data: Discount Curve***

In [4]:
start_date = datetime(2007, 3, 30)
end_date = datetime(2024, 4, 30)
currency = "USD"
df_rates_type = "OIS"
data_reqeust = DFCurveRequest(start_date, end_date, currency, df_rates_type)

In [5]:
data_set = JPDFCurveDataSource().initialize(data_reqeust)
df_curve_start = data_set.get_spot_rate_curves(start_date)
df_curve_end = data_set.get_spot_rate_curves(end_date)
pd.DataFrame().from_dict([df_curve_start, df_curve_end]).sort_index(axis=1)

,0.002740,0.083333,0.166667,0.250000,0.333333,0.416667,0.500000,0.583333,0.666667,0.750000,0.833333,0.916667,1.000000,1.083333,1.166667,1.250000,1.333333,1.416667,1.500000,1.583333,1.666667,1.750000,1.833333,1.916667,2.000000,2.250000,2.500000,2.750000,3.000000,3.250000,3.500000,3.750000,4.000000,4.250000,4.500000,4.750000,5.000000,6.000000,7.000000,8.000000,9.000000,10.000000,15.000000,20.000000,25.000000,30.000000,40.000000,50.000000,60.000000
0,5.249990,5.251973,5.260705,5.269648,5.259854,5.258202,5.253512,5.237950,5.218145,5.197144,5.175650,5.153625,5.133118,5.096237,5.062361,5.030949,5.004232,4.981538,4.962003,4.946784,4.933951,4.921924,4.912335,4.905481,4.900069,4.867799,4.850105,4.843753,4.846411,4.839644,4.840748,4.84735,4.857670,4.858154,4.862871,4.871209,4.882853,4.916600,4.953785,4.991983,5.029268,5.062949,5.200160,5.260161,5.273328,5.274276,5.260972,5.262621,5.263551
1,5.326784,5.339019,5.343378,5.349938,5.350758,5.349174,5.344649,5.338872,5.331004,5.320636,5.307517,5.294512,5.281262,5.242286,5.209668,5.177219,5.146381,5.119497,5.090870,5.066743,5.042278,5.018725,4.998393,4.977942,4.957856,4.878157,4.814978,4.764796,4.724619,4.671244,4.627545,4.59295,4.565881,4.528932,4.499739,4.477752,4.461716,4.395647,4.349522,4.316377,4.293636,4.277373,4.252709,4.198280,4.091791,3.983498,3.756570,3.524635,3.363696


***Data: Forward Rates***

In [6]:
start_date = datetime(2007, 3, 30)
end_date = datetime(2024, 4, 30)
currency = "USD"
fwd_rate_request = ForwardRateRequest(start_date, end_date, currency)

In [7]:
fwd_rate_data = JPForwardRateDataSource().initialize(fwd_rate_request)
fwd_rate_start = fwd_rate_data.get_forward_rate_curves(start_date)
pd.DataFrame(fwd_rate_start).sort_index().sort_index(axis=1)

,10y,12y,15y,18m,1m,1y,20y,25y,2y,30y,3m,3y,4y,5y,6m,6y,7y,8y,9y
0.019178,5.177241,5.242387,5.311936,5.079420,5.329855,5.225185,5.371449,5.391038,4.991446,5.393100,5.346534,4.944374,4.959340,4.989787,5.328018,5.025678,5.066482,5.104238,5.141882
0.038356,5.177316,5.242428,5.312046,5.074057,5.330370,5.219005,5.371454,5.390971,4.987411,5.393011,5.340559,4.941796,4.957895,4.988858,5.320085,5.025074,5.065868,5.103897,5.141861
0.057534,5.177208,5.242499,5.312065,5.064557,5.328615,5.207025,5.371445,5.390906,4.980536,5.392915,5.332034,4.938161,4.955767,4.987479,5.310605,5.024246,5.065359,5.103592,5.141723
0.083333,5.177586,5.243018,5.312586,5.061609,5.325453,5.193091,5.371898,5.391314,4.971916,5.393279,5.321418,4.934182,4.953379,4.986066,5.295624,5.023597,5.065149,5.103913,5.141957
0.166667,5.177058,5.243016,5.312294,5.012822,5.323677,5.138155,5.371291,5.390501,4.943999,5.392348,5.292283,4.918778,4.943661,4.979792,5.243060,5.019936,5.062620,5.101849,5.140795
0.250000,5.177548,5.243714,5.313051,4.975181,5.242078,5.073382,5.371696,5.390626,4.912522,5.392312,5.248712,4.902764,4.933871,4.973812,5.181591,5.016414,5.060783,5.101001,5.140672
0.500000,5.180501,5.246986,5.315590,4.853178,5.101209,4.920800,5.372826,5.390842,4.843102,5.391984,5.046775,4.869201,4.913680,4.962546,4.974820,5.011375,5.058122,5.100841,5.142272
0.750000,5.190964,5.256693,5.323607,4.781363,4.875631,4.812185,5.378543,5.395165,4.804471,5.395461,4.839657,4.856412,4.909279,4.964431,4.803032,5.017551,5.065715,5.109335,5.151934
1.000000,5.207444,5.272523,5.336267,4.768881,4.734142,4.748449,5.388764,5.403292,4.792524,5.402553,4.708802,4.860091,4.921258,4.977129,4.704035,5.032392,5.080647,5.126658,5.168093
1.500000,5.254535,5.314365,5.371340,4.795476,4.625593,4.761053,5.416137,5.426221,4.841254,5.422701,4.630754,4.910792,4.973895,5.031859,4.660879,5.084773,5.131269,5.176273,5.216738


***Data: Rate Vol***

In [8]:
start_date = datetime(2007, 3, 30)
end_date = datetime(2024, 4, 30)
currency = "USD"
rate_vol_request = RateVolRequest(start_date, end_date, currency)

In [9]:
rate_vol_data = JPRateVolDataSource().initialize(rate_vol_request)

In [10]:
rate_vol_end = rate_vol_data.get_vol_cube(end_date)
rate_vol_end_1y2y = rate_vol_end['2y'][1]
pd.DataFrame().from_dict([rate_vol_end_1y2y]).sort_index(axis=1)

,-3.0,-1.0,-0.5,0.5,1.0,3.0
0,1.496765,1.321788,1.272261,1.231668,1.249803,1.459299
